# Preprocessing Encoder, Pipeline, SequentialEncoder and FeatureUnion example

The easyflow.preprocessing module contains functionality similar to what sklearn does with its Pipeline, FeatureUnion and ColumnTransformer does. 

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Normalization, IntegerLookup, StringLookup

In [2]:
# local imports
from easyflow.data.mapper import TensorflowDataMapper
from easyflow.preprocessing.preprocessor import Encoder, Pipeline, SequentialEncoder, FeatureUnion
from easyflow.preprocessing.custom import IdentityPreprocessingLayer

## Read in data and map as tf.data.Dataset

Use the TensorflowDataMapper class to map pandas data frame to a tf.data.Dataset type

In [3]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
dataframe = pd.read_csv(file_url)
labels = dataframe.pop("target")

batch_size = 32
dataset_mapper = TensorflowDataMapper() 
dataset = dataset_mapper.map(dataframe, labels)
train_data_set, val_data_set = dataset_mapper.split_data_set(dataset)
train_data_set = train_data_set.batch(batch_size)
val_data_set = val_data_set.batch(batch_size)

In [4]:
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal


## Set constants

In [5]:
NUMERICAL_FEATURES = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope']
CATEGORICAL_FEATURES = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'ca']
# thal is represented as a string
STRING_CATEGORICAL_FEATURES = ['thal']

## Setup Preprocessing layer using FeatureUnion

Use Encoder and SequentialEncoder to preprocess features by putting everything in a FeatureUnion object

In [7]:
feature_encoder_list = [
                        ('numeric_encoder', Normalization(), NUMERICAL_FEATURES),
                        ('categorical_encoder', IntegerLookup(output_mode='binary'), CATEGORICAL_FEATURES),
                        # For feature thal we first need to run StringLookup followed by a IntegerLookup layer
                        ('string_encoder', [StringLookup(), IntegerLookup(output_mode='binary')], STRING_CATEGORICAL_FEATURES)
                        ]


In [8]:
encoder = FeatureUnion(feature_encoder_list)
all_feature_inputs, preprocessing_layer = encoder.encode(dataset)

In [9]:
print(preprocessing_layer)

KerasTensor(type_spec=TensorSpec(shape=(None, 30), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")


In [9]:
# setup simple network
x = tf.keras.layers.Dense(128, activation="relu")(preprocessing_layer)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=all_feature_inputs, outputs=outputs)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy'), tf.keras.metrics.AUC(name='auc')])

In [1]:
history=model.fit(train_data_set, validation_data=val_data_set, epochs=10)

In [11]:
model.save('simple_model')

INFO:tensorflow:Assets written to: preprocessing_model/assets


In [12]:
loaded_model = tf.keras.models.load_model("simple_model")

In [14]:
loaded_model.predict(val_data_set.take(1))

array([[0.0971629 ],
       [0.10495132],
       [0.06081408],
       [0.6935728 ],
       [0.08850741],
       [0.24280909],
       [0.03417057],
       [0.2560429 ],
       [0.5533061 ],
       [0.6134831 ],
       [0.3023044 ],
       [0.07778424],
       [0.0370003 ],
       [0.9218978 ],
       [0.862098  ],
       [0.64490104],
       [0.1212824 ],
       [0.09255898],
       [0.25355804],
       [0.0864307 ],
       [0.22723436],
       [0.2830603 ],
       [0.13806298],
       [0.5916211 ],
       [0.07507512],
       [0.06467262],
       [0.44774485],
       [0.07877955],
       [0.04439986],
       [0.05433959],
       [0.5220138 ],
       [0.71563536]], dtype=float32)

## More flexibility with Pipeline

FeatureUnion subclasses Pipeline and concatenates(i.e union) the layers. For more flexibility like a wide and deep neural network, Pipeline class will give you more flexibility in that sense

In [12]:
feature_encoder_list = [
                        ('numeric_encoder', Normalization(), NUMERICAL_FEATURES),
                        ('categorical_encoder', IntegerLookup(output_mode='binary'), CATEGORICAL_FEATURES),
                        # For feature thal we first need to run StringLookup followed by a IntegerLookup layer
                        ('string_encoder', [StringLookup(), IntegerLookup(output_mode='binary')], STRING_CATEGORICAL_FEATURES)
                        ]

encoder = Pipeline(feature_encoder_list)
all_feature_inputs1, preprocessing_layer1 = encoder.encode(dataset)